## Assistants API 개요


새로운 [Assistants API](https://platform.openai.com/docs/assistants/overview)는 [Chat Completions API](https://platform.openai.com/docs/guides/text-generation/chat-completions-api)를 발전시킨 것으로, 개발자가 어시스턴트와 유사한 경험을 간편하게 만들고 코드 해석기 및 검색과 같은 강력한 도구에 액세스할 수 있도록 하기 위한 것입니다.


### Assistants API 의 탄생 배경

**Chat Completions API**의 기본 요소는 `Messages`이며, 여기에 `Model`(`gpt-3.5-turbo`, `gpt-4-turbo-preview` 등)을 사용하여 `Completion`을 수행합니다.

`Chat Completions API` 는 메시지를 주고 받는데에는 가볍게 잘 동작하지만, 상태를 관리할 수는 없습니다. 우리는 이것을 **Stateless** 하다는 표현을 사용합니다.

여기서 상태란, ChatGPT 와 플러그인 형식으로 동작하는 도구의 개념으로 이해하셔도 좋습니다.

예를 들면, '검색(search)', '문서 검색(retrieval)', '코드 실행(code interpreter)' 등의 기능이 부재하기 때문에, 이에 대한 보완책으로 `Assistants API` 가 탄생하게 되었습니다.


### Assistants API\*\*의 기본 요소

![Assistants API Diagram](https://teddylee777.github.io/2024-02-12-openai-assistant-tutorial/assistants_overview_diagram.png)

- `Assistants`, 기본 모델, 지침, 도구 및 (문맥) 문서를 캡슐화합니다,
- `Threads`, 대화의 상태를 나타냅니다,
- `Runs`, `Assistant`의 `Thread`에서의 실행을 구동하며, 텍스트 응답 및 다단계 도구 사용을 포함합니다.

우리는 이것들을 사용하여 강력하고 상태가 있는(stateful) 경험을 만드는 방법을 살펴볼 것입니다.

## 설정

### Python 라이브러리 설정

> **참고**
> OpenAI는 Assistants API를 지원하기 위해 [Python 라이브러리](https://github.com/openai/openai-python)를 업데이트했습니다. 따라서 최신 버전으로 업데이트 한 뒤 튜토리얼을 진행할 것을 권장합니다.


이 문서는 `openai` 라이브러리를 최신 버전으로 업그레이드하고 설치하는 방법을 설명합니다. `pip` 명령어를 사용하여 Python 환경에 `openai` 라이브러리를 설치하며, 이는 OpenAI의 API를 활용하는 데 필요한 기본적인 단계입니다.


In [ ]:
# openai 라이브러리를 최신 버전으로 업그레이드하여 설치합니다.
!pip install --upgrade openai -q

그리고 다음을 실행하여 최신 상태인지 확인하세요:


이 명령어는 `openai` 패키지의 버전 정보를 확인하기 위해 사용됩니다. `pip show` 명령어를 통해 특정 패키지에 대한 상세 정보를 출력하고, `grep` 명령어를 사용하여 그 중 'Version'에 해당하는 줄만 필터링하여 출력합니다.


In [1]:
!pip show openai | grep Version # openai 패키지의 버전 정보를 확인합니다.

Version: 1.12.0


### Pretty Printing Helper


`show_json` 함수는 인자로 받은 객체의 모델을 JSON 형태로 변환하여 출력합니다. 이 함수는 객체의 `model_dump_json` 메서드를 호출하여 JSON 문자열을 얻고, 이를 `json.loads`를 통해 파싱한 후, `display` 함수를 사용하여 결과를 출력합니다.


In [ ]:
import json


def show_json(obj):
    # obj의 모델을 JSON 형태로 변환한 후 출력합니다.
    display(json.loads(obj.model_dump_json()))

## Assistants API를 사용한 흐름 이해하기


### 보조자 (Assistant)


Assistants API를 시작하는 가장 쉬운 방법은 [Assistants Playground](https://platform.openai.com/playground)를 통하는 것입니다.

![Assistants Playground](./images/assistants_overview_assistants_playground.png)

우리는 보조자를 만들기 시작합시다! 우리의 [문서](https://platform.openai.com/docs/assistants/overview)에서와 같이 Math Tutor를 만들겠습니다.

![Creating New Assistant](./images/assistants_overview_new_assistant.png)

생성한 Assistant를 [Assistants Dashboard](https://platform.openai.com/assistants)에서 볼 수 있습니다.

![Assistants Dashboard](./images/assistants_overview_assistants_dashboard.png)

Assistants API를 통해 직접 Assistant를 생성할 수도 있습니다, 다음과 같이:


이 코드는 OpenAI의 GPT-4 모델을 사용하여 'Math Tutor'라는 이름의 개인 수학 과외 선생님 역할을 하는 챗봇을 생성합니다. 사용자는 환경 변수 `OPENAI_API_KEY`를 통해 API 키를 제공하거나, 환경 변수가 설정되지 않은 경우 코드 내에서 직접 키를 입력할 수 있습니다. 생성된 챗봇은 간단한 문장이나 한 문장 이하로 질문에 답변하도록 지시받습니다. 마지막으로, `show_json` 함수를 통해 생성된 챗봇의 정보를 JSON 형태로 출력합니다.


In [ ]:
# API KEY 정보를 불러옵니다
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from openai import OpenAI
import os

# os.environ["OPENAI_API_KEY"] = "API KEY를 입력해 주세요"
# OPENAI_API_KEY 를 설정합니다.
api_key = os.environ.get("OPENAI_API_KEY")

In [ ]:
# OpenAI API를 사용하기 위한 클라이언트 객체를 생성합니다.
client = OpenAI(api_key=api_key)

# 수학 과외 선생님 역할을 하는 챗봇을 생성합니다.
# 이 챗봇은 간단한 문장이나 한 문장으로 질문에 답변합니다.
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-4-turbo-preview",
)
# 생성된 챗봇의 정보를 JSON 형태로 출력합니다.
show_json(assistant)

대시보드를 통해 Assistant를 생성하든 API를 사용하든, `Assistant ID` 를 추적하는 것이 중요합니다. 이것은 Threads와 Runs 전반에 걸쳐 Assistant를 참조하는 방법입니다.


다음으로, 새로운 Thread를 생성하고 그 안에 Message를 추가할 것입니다. 이것은 우리 대화의 상태를 유지할 것이므로, 매번 전체 메시지 기록을 다시 보내지 않아도 됩니다.


### 스레드


새로운 대화 스레드를 생성해 보겠습니다.


이 함수는 `client` 객체의 `beta` 속성에 접근하여 `threads.create()` 메서드를 호출함으로써 새로운 스레드를 생성합니다. 생성된 스레드의 정보는 `show_json` 함수를 통해 JSON 형식으로 출력됩니다. 이 과정은 특히 API를 통한 자원 생성 및 결과 확인에 유용합니다.


In [ ]:
thread = client.beta.threads.create()
# 새로운 스레드를 생성합니다.
show_json(thread)
# 생성된 스레드의 정보를 JSON 형식으로 출력합니다.

다음은 스레드에 메시지를 추가해 보도록 하겠습니다.


이 함수는 클라이언트의 베타 API를 사용하여 특정 스레드에 메시지를 생성합니다. `thread_id`는 메시지가 속할 스레드의 ID를 지정하며, `role`은 메시지를 보내는 사용자의 역할을 나타냅니다. 여기서는 `"user"`로 설정되어 있습니다. `content`는 전송할 메시지의 내용을 포함합니다. 이 예제에서는 `"I need to solve the equation `3x + 11 = 14`. Can you help me?"`라는 수학 문제 해결을 요청하는 메시지입니다. 마지막으로 `show_json` 함수는 생성된 메시지 객체를 JSON 형식으로 출력합니다.


In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)
show_json(message)

> **참고**
> 전체 대화 기록을 매번 보내지 않더라도, 각 실행마다 전체 대화 기록의 토큰에 대해 여전히 요금이 부과됩니다.


### 실행

우리가 만든 **Thread**가 이전에 만든 Assistant와 연결되어 있지 **않다는** 것을 주목하세요! Thread는 Assistant와 독립적으로 존재하며, 만약 여러분이 ChatGPT(모델/GPT에 thread가 연결되어 있는 경우)를 사용해 본 적이 있다면 예상과 다를 수 있습니다.

주어진 Thread에 대해 Assistant로부터 완성을 얻으려면 Run을 생성해야 합니다. Run을 생성하면 Assistant에게 Thread의 메시지를 살펴보고 조치를 취하라는 지시를 합니다: 단일 응답을 추가하거나 도구를 사용하는 것입니다.

> **참고**
> Run은 Assistants API와 Chat Completions API 사이의 주요 차이점입니다. Chat Completions에서는 모델이 단일 메시지로만 응답하지만, Assistants API에서는 Run이 하나 또는 여러 도구를 사용하고, Thread에 여러 메시지를 추가할 수 있습니다.

사용자에게 응답하도록 Assistant를 활성화하려면 Run을 생성합시다. 앞서 언급했듯이, Assistant와 Thread _둘 다_ 지정해야 합니다.


이 함수는 `client` 객체의 `beta` 속성 내에 위치한 `threads` 객체의 `runs` 속성을 사용하여 새로운 실행(run)을 생성합니다. 여기서 `thread_id`와 `assistant_id`는 각각 스레드와 어시스턴트의 식별자로, 이들의 `id` 속성을 통해 전달됩니다. 생성된 실행 객체는 `run` 변수에 할당되며, `show_json(run)` 함수를 통해 JSON 형태로 출력됩니다.


In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,  # 생성한 스레드 ID
    assistant_id=assistant.id,  # 적용할 Assistant ID
)
show_json(run)

Chat Completions API에서 완성을 생성하는 것과 달리, **Run을 생성하는 것은 비동기 작업입니다**. 이는 Run의 메타데이터와 함께 즉시 반환되며, 여기에는 처음에 `queued`로 설정될 `status`가 포함됩니다. `status`는 Assistant가 작업을 수행함에 따라(도구 사용 및 메시지 추가와 같은) 업데이트될 것입니다.

Assistant가 처리를 완료했는지 알기 위해서는 Run을 반복해서 폴링할 수 있습니다. (스트리밍 지원이 곧 제공될 예정입니다!) 여기서는 `queued` 또는 `in_progress` 상태만 확인하고 있지만, 실제로 Run은 [다양한 상태 변경](https://platform.openai.com/docs/api-reference/runs/object#runs/object-status)을 겪을 수 있으며, 이를 사용자에게 표시하기로 선택할 수 있습니다. (이것들은 단계라고 하며, 나중에 다룰 예정입니다.)


이 함수는 특정 실행(`run`)이 완료될 때까지 대기하며, 실행 상태를 주기적으로 확인합니다. `run` 객체의 상태가 `"queued"` 또는 `"in_progress"`인 경우, 해당 실행의 최신 상태를 검색하여 업데이트합니다. 상태 확인 사이에는 0.5초의 휴식 시간을 두어 API 요청의 부하를 줄입니다. 이 과정은 실행이 완료될 때까지 (`"completed"` 상태가 될 때까지) 반복됩니다.


In [ ]:
import time


def wait_on_run(run, thread):
    # 주어진 실행(run)이 완료될 때까지 대기합니다.
    while run.status == "queued" or run.status == "in_progress":
        # 실행 상태를 최신 정보로 업데이트합니다.
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        # API 요청 사이에 짧은 휴식을 취합니다.
        time.sleep(0.5)
    return run

이 함수들은 `run` 객체의 실행을 관리하고, 그 결과를 JSON 형식으로 출력합니다. `wait_on_run` 함수는 `run` 객체가 주어진 `thread`에서 실행을 완료할 때까지 대기하도록 합니다. 실행이 완료되면, `show_json` 함수를 사용하여 `run` 객체의 정보를 JSON 형식으로 출력합니다.


In [ ]:
run = wait_on_run(run, thread)
# run 객체를 대기 상태로 설정하고, 해당 스레드에서 실행을 완료할 때까지 기다립니다.
show_json(run)
# run 객체의 정보를 JSON 형식으로 출력합니다.

### 메시지


Run이 완료되었으므로, Assistant에 의해 추가된 내용을 보기 위해 Thread에서 Messages를 나열할 수 있습니다.


이 함수는 `client` 객체의 `beta` 버전 API를 사용하여 특정 스레드(`thread.id`)에 속한 메시지 목록을 조회합니다. 조회된 메시지 목록은 `show_json` 함수를 통해 JSON 형식으로 출력됩니다. 이 과정은 API를 통한 데이터 조회와 결과의 시각화를 포함하며, Python에서 REST API 호출과 응답 처리를 다루는 기본적인 예시를 보여줍니다.


In [ ]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
# thread.id를 사용하여 메시지 목록을 가져옵니다.
show_json(messages)

보시다시피, 메시지는 역순으로 정렬됩니다 - 이는 가장 최근의 결과가 항상 첫 번째 `page`에 있도록 하기 위해서입니다(결과는 페이지별로 나뉠 수 있습니다). 이 점을 주의하세요, 왜냐하면 이는 Chat Completions API에서 메시지의 순서와 반대이기 때문입니다.


조금 더 자세히 설명해 달라고 요청해 봅시다!


이 문서는 클라이언트를 사용하여 스레드에 메시지를 추가하고, 실행을 시작한 후, 실행이 완료될 때까지 대기하는 과정을 설명합니다. 마지막으로, 특정 사용자 메시지 이후에 스레드에 추가된 모든 메시지를 검색하는 방법을 보여줍니다. 사용자는 `client.beta.threads.messages.create` 함수를 호출하여 스레드에 메시지를 추가하고, `client.beta.threads.runs.create` 함수를 사용하여 실행을 시작합니다. `wait_on_run` 함수는 실행이 완료될 때까지 대기하며, `client.beta.threads.messages.list` 함수는 지정된 조건에 맞는 메시지들을 검색합니다. 마지막으로, `show_json` 함수는 검색된 메시지들을 JSON 형태로 출력합니다.


In [ ]:
# 스레드에 추가할 메시지 생성
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="Could you explain this to me in Korean?"
)

# 실행을 시작함
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# 완료될 때까지 대기
wait_on_run(run, thread)

# 마지막 사용자 메시지 이후에 추가된 모든 메시지를 검색
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)
show_json(messages)

이것은 응답을 받기 위해 많은 단계를 거치는 것처럼 느껴질 수 있습니다, 특히 이 간단한 예제의 경우에는 더욱 그렇습니다. 하지만, 곧 우리는 코드를 거의 변경하지 않고도 우리의 Assistant에 매우 강력한 기능을 추가할 수 있는 방법을 보게 될 것입니다!


### 예시


이 모든 것을 어떻게 결합할 수 있는지 살펴보겠습니다. 아래는 여러분이 생성한 Assistant를 사용하기 위해 필요한 모든 코드입니다.

우리가 이미 우리의 수학 Assistant를 생성했기 때문에, `MATH_ASSISTANT_ID`에 그 ID를 저장했습니다. 그런 다음 두 가지 함수를 정의했습니다:

- `submit_message`: Thread에 Message를 생성하고, 새로운 Run을 시작(및 반환)합니다.
- `get_response`: Thread에서 Message의 목록을 반환합니다.


이 모듈은 OpenAI API를 사용하여 수학 보조 프로그램과의 대화를 관리합니다. `OpenAI` 클래스를 사용하여 API 클라이언트를 초기화하고, `submit_message` 함수를 통해 사용자 메시지를 보조 프로그램에 제출합니다. 이후, `get_response` 함수를 사용하여 보조 프로그램의 응답을 가져옵니다. 사용자와 보조 프로그램 간의 대화는 `thread` 객체를 통해 관리됩니다.


In [ ]:
from openai import OpenAI

MATH_ASSISTANT_ID = assistant.id  # OpenAI에서 제공하는 수학 보조 프로그램의 ID

# OpenAI API를 사용하기 위한 클라이언트 객체를 생성합니다.
client = OpenAI(api_key=api_key)


def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

저는 또한 재사용할 수 있는 `create_thread_and_run` 함수를 정의했습니다(사실상 우리 API의 [`client.beta.threads.create_and_run`](https://platform.openai.com/docs/api-reference/runs/createThreadAndRun) 복합 함수와 거의 동일합니다 ;) ). 마지막으로, 우리는 각각의 가상 사용자 요청을 새로운 스레드에 제출할 수 있습니다.

이러한 API 호출이 모두 비동기 작업임에 주목하세요; 이는 실제로 `asyncio`와 같은 비동기 라이브러리를 사용하지 않고도 코드에서 비동기 동작을 얻을 수 있음을 의미합니다!


이 함수는 사용자의 입력을 받아 새로운 스레드를 생성하고, 해당 스레드에서 수학 도우미에게 메시지를 제출하는 기능을 수행합니다. `create_thread_and_run` 함수는 사용자의 질문을 인자로 받아, 클라이언트의 베타 API를 통해 스레드를 생성(`client.beta.threads.create()`)하고, `submit_message` 함수를 사용하여 수학 도우미(`MATH_ASSISTANT_ID`)에게 해당 스레드와 함께 메시지를 제출합니다. 이 과정은 동시에 여러 사용자의 요청을 처리할 수 있도록 설계되었습니다. 예시로, 세 가지 다른 사용자 입력을 처리하는 코드가 포함되어 있으며, 이는 동시에 여러 요청을 에뮬레이션하는 데 사용됩니다. 각 요청은 수학 문제 해결, 선형 대수학 설명 요청, 수학에 대한 불만을 표현하는 메시지로 구성됩니다.


In [ ]:
def create_thread_and_run(user_input):
    # 사용자 입력을 받아 새로운 스레드를 생성하고, 수학 도우미에게 메시지를 제출합니다.
    thread = client.beta.threads.create()
    run = submit_message(MATH_ASSISTANT_ID, thread, user_input)
    return thread, run


# 동시에 여러 사용자 요청을 에뮬레이션합니다.
thread1, run1 = create_thread_and_run(
    "I need to solve the equation `3x + 11 = 14`. Can you help me?"
)
thread2, run2 = create_thread_and_run(
    "Could you explain linear algebra to me?")
thread3, run3 = create_thread_and_run("I don't like math. What can I do?")

# 이제 모든 실행이 진행 중입니다...

모든 실행이 진행되고 나면, 각각을 기다린 후 응답을 받을 수 있습니다.


이 함수들은 비동기 작업의 상태를 확인하고, 완료되면 결과를 예쁘게 출력하는 기능을 제공합니다. `pretty_print` 함수는 메시지 목록을 받아 각 메시지의 역할과 내용을 출력합니다. `wait_on_run` 함수는 특정 작업(`run`)이 완료될 때까지 대기하며, 작업 상태가 `queued` 또는 `in_progress`인 경우 주기적으로 상태를 확인합니다. 이후, 세 개의 다른 작업(`run1`, `run2`, `run3`)에 대해 이 함수를 사용하여 각각의 작업이 완료될 때까지 대기하고, 완료된 후에는 `pretty_print` 함수를 통해 결과를 출력합니다. 마지막으로, `run4`를 통해 세 번째 스레드에 있는 조수에게 감사의 메시지를 보내고, 이 작업의 완료를 기다린 후 결과를 출력합니다.


In [ ]:
import time


# 메시지를 예쁘게 출력하는 도우미 함수
def pretty_print(messages):
    print("# 메시지들")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


# 반복문에서 대기하는 함수
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


# 첫 번째 실행을 위해 대기
run1 = wait_on_run(run1, thread1)
pretty_print(get_response(thread1))

# 두 번째 실행을 위해 대기
run2 = wait_on_run(run2, thread2)
pretty_print(get_response(thread2))

# 세 번째 실행을 위해 대기
run3 = wait_on_run(run3, thread3)
pretty_print(get_response(thread3))

# 세 번째 스레드에서 조수에게 감사 인사하기 :)
run4 = submit_message(MATH_ASSISTANT_ID, thread3, "Thank you!")
run4 = wait_on_run(run4, thread3)
pretty_print(get_response(thread3))

당신은 아마 이 코드가 우리의 수학 보조기구에만 특화된 것이 아니라는 것을 알아챘을 겁니다... 이 코드는 보조기구 ID만 변경하면 어떤 새로운 보조기구에 대해서도 작동합니다! 이것이 Assistants API의 힘입니다.


### Code Interpreter (템플릿 코드)


In [ ]:
# API KEY 정보를 불러옵니다
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import os
import json

# os.environ["OPENAI_API_KEY"] = "API KEY를 입력해 주세요"
# OPENAI_API_KEY 를 설정합니다.
api_key = os.environ.get("OPENAI_API_KEY")


def show_json(obj):
    # obj의 모델을 JSON 형태로 변환한 후 출력합니다.
    display(json.loads(obj.model_dump_json()))

1. Assistant 생성
   1. [Assistants Playground](https://platform.openai.com/playground) 에서 이미 Assistant 를 생성한 경우
   2. Assistant 를 생성하지 않은 경우


In [ ]:
# 1-1. Assistant ID를 불러옵니다(Playground에서 생성한 Assistant ID)
ASSISTANT_ID = "asst_V8s4Ku4Eiid5QC9WABlwDsYs"

In [ ]:
# 1-2. Assistant 를 생성합니다.
from openai import OpenAI

# OpenAI API를 사용하기 위한 클라이언트 객체를 생성합니다.
client = OpenAI(api_key=api_key)

# 수학 과외 선생님 역할을 하는 챗봇을 생성합니다.
# 이 챗봇은 간단한 문장이나 한 문장으로 질문에 답변합니다.
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-4-turbo-preview",
)
# 생성된 챗봇의 정보를 JSON 형태로 출력합니다.
show_json(assistant)
ASSISTANT_ID = assistant.id

2. 스레드(Thread) 생성하기
   1. 스레드를 이미 생성한 경우
   2. 스레드를 새롭게 생성하는 경우


In [ ]:
# 2-1. 스레드를 이미 생성한 경우
THREAD_ID = "thread_6We5fHvb5NBuacPfZYkqUWlO"

In [ ]:
# 2-2. 스레드를 새롭게 생성합니다.
thread = client.beta.threads.create()
# 새로운 스레드를 생성합니다.
show_json(thread)
# 새롭게 생성한 스레드 ID를 저장합니다.
THREAD_ID = thread.id

3. 스레드에 메시지 생성
   1. 스레드에 새로운 메시지를 추가 합니다.
   2. 스레드를 실행(run) 합니다.
   3. 스레드의 상태를 확인합니다.(대기중, 작업중, 완료, etc)
   4. 스레드에서 최신 메시지를 조회한 뒤 결과를 확인합니다.


In [ ]:
import time


# 반복문에서 대기하는 함수
def wait_on_run(run, thread_id):
    while run.status == "queued" or run.status == "in_progress":
        # 3-3. 실행 상태를 최신 정보로 업데이트합니다.
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


def submit_message(assistant_id, thread_id, user_message):
    # 3-1. 스레드에 종속된 메시지를 '추가' 합니다.
    client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=user_message
    )
    # 3-2. 스레드를 실행합니다.
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
    )
    return run


def get_response(thread_id):
    # 3-4. 스레드에 종속된 메시지를 '조회' 합니다.
    return client.beta.threads.messages.list(thread_id=thread_id, order="asc")


def print_message(response):
    for res in response:
        print(f"[{res.role.upper()}]\n{res.content[0].text.value}\n")


def ask(assistant_id, thread_id, user_message):
    run = submit_message(
        assistant_id,
        thread_id,
        user_message,
    )
    # 실행이 완료될 때까지 대기합니다.
    run = wait_on_run(run, thread_id)
    print_message(get_response(thread_id).data[-2:])
    return run

In [ ]:
run = ask(ASSISTANT_ID, THREAD_ID, "I need to solve `1 + 20`. Can you help me?")

In [ ]:
# 전체 대화내용 출력
print_message(get_response(THREAD_ID).data[:])

## 도구

Assistants API의 핵심 기능 중 하나는 Code Interpreter, Retrieval, 그리고 사용자 정의 함수와 같은 도구로 우리의 Assistants를 장착할 수 있는 능력입니다. 각각에 대해 살펴보겠습니다.

### Code Interpreter

우리의 Math Tutor에 [Code Interpreter](https://platform.openai.com/docs/assistants/tools/code-interpreter) 도구를 장착해 보겠습니다.


![Enabling code interpreter](./images/assistants_overview_enable_code_interpreter.png)


이는 대시보드에서 할 수 있기도 하고 또는 Assistant API 를 사용해서 추가할 수도 있습니다.


이 함수는 `client.beta.assistants.update`를 사용하여 특정 도우미(assistant)의 설정을 업데이트합니다. 여기서 `MATH_ASSISTANT_ID`는 업데이트할 도우미의 식별자(ID)입니다. `tools` 매개변수는 도우미에게 추가할 도구들의 목록을 지정하며, 이 예제에서는 `code_interpreter` 타입의 도구 하나를 추가합니다. 마지막으로, `show_json` 함수는 업데이트된 도우미의 정보를 JSON 형식으로 출력합니다.


In [ ]:
assistant = client.beta.assistants.update(
    ASSISTANT_ID,
    tools=[{"type": "code_interpreter"}],
)
show_json(assistant)

이제, Assistant에게 새로운 도구를 사용하도록 요청합시다.


이 함수들은 피보나치 수열의 첫 20개 숫자를 생성하고, 이를 예쁘게 출력하는 과정을 단계별로 수행합니다. `create_thread_and_run` 함수는 코드 실행을 위한 스레드를 생성하고 실행합니다. `wait_on_run` 함수는 주어진 스레드의 실행이 완료될 때까지 대기합니다. 마지막으로, `pretty_print` 함수는 스레드에서 받은 응답을 예쁘게 출력합니다.


In [ ]:
run = ask(ASSISTANT_ID, THREAD_ID, "Generate the first 20 fibbonaci numbers with code.")

그게 전부입니다! 보조기구는 배경에서 코드 해석기를 사용하고, 우리에게 최종 응답을 주었습니다.

일부 사용 사례에서는 이것으로 충분할 수 있습니다. 하지만, 우리가 보조기구가 정확히 무엇을 하는지에 대한 더 많은 세부 정보를 원한다면, 우리는 실행의 단계들을 살펴볼 수 있습니다.

### 단계들


Run은 하나 이상의 Step으로 구성됩니다. Run과 마찬가지로, 각 Step은 조회할 수 있는 `status`를 가지고 있습니다. 이는 사용자에게 Step의 진행 상황을 표면화하는 데 유용합니다 (예: Assistant가 코드를 작성하거나 검색을 수행하는 동안 스피너).


이 함수는 클라이언트의 베타 API를 사용하여 특정 스레드와 실행 ID에 대한 실행 단계 목록을 오름차순(`asc`)으로 검색합니다. 여기서 `client.beta.threads.runs.steps.list` 메소드는 `thread_id`와 `run_id`를 인자로 받아 해당 조건에 맞는 실행 단계들을 반환합니다. 이는 AI 프로그래밍에서 실행 단계의 순서와 진행 상황을 추적하는 데 유용하게 사용될 수 있습니다.


In [ ]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=THREAD_ID, run_id=run.id, order="asc"
)

각 단계의 `step_details`를 살펴봅시다.

단계별로 생각해 봅시다.


이 함수는 `run_steps.data`에서 각 단계(`step`)를 순회하며, 해당 단계의 세부 정보(`step_details`)를 JSON 형식으로 출력합니다. 여기서 `show_json` 함수는 세부 정보를 JSON으로 변환하는 역할을 하며, `json.dumps` 함수는 이를 문자열로 변환하여 출력합니다. 출력 시 들여쓰기는 4칸을 사용하여 가독성을 높입니다.


In [ ]:
for step in run_steps.data:
    # 각 단계의 세부 정보를 가져옵니다.
    step_details = step.step_details
    # 세부 정보를 JSON 형식으로 출력합니다.
    show_json(step_details)

두 단계의 `step_details`를 볼 수 있습니다:

1. `tool_calls` (단수가 아닌 복수형이며, 하나의 단계에서 하나 이상이 될 수 있습니다)
2. `message_creation`

첫 번째 단계는 `tool_calls`이며, 특히 `code_interpreter`를 사용하는데, 이는 다음을 포함합니다:

- `input`, 이는 도구가 호출되기 전에 생성된 Python 코드였으며,
- `output`, 이는 Code Interpreter를 실행한 결과였습니다.

두 번째 단계는 `message_creation`이며, 사용자에게 결과를 전달하기 위해 스레드에 추가된 `message`를 포함합니다.


### 검색

Assistants API에서 또 다른 강력한 도구는 [검색](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval)입니다: 질문에 답변할 때 보조기가 지식 기반으로 사용할 파일을 업로드할 수 있는 기능입니다. 이 기능은 대시보드나 API에서도 활성화할 수 있으며, 사용하고자 하는 파일을 업로드할 수 있습니다.


![Enabling retrieval](./images/assistants_overview_enable_retrieval.png)


이 코드는 클라이언트를 사용하여 파일을 업로드하고, 업로드된 파일을 사용하여 보조기구를 업데이트하는 과정을 보여줍니다. 먼저, `client.files.create` 함수를 호출하여 PDF 파일을 업로드합니다. 이때, 파일의 용도는 `assistants`로 지정됩니다. 그 다음, `client.beta.assistants.update` 함수를 사용하여 보조기구를 업데이트합니다. 여기서는 `MATH_ASSISTANT_ID`를 사용하여 특정 보조기구를 식별하고, `tools` 배열을 통해 `code_interpreter`와 `retrieval` 도구를 추가합니다. 마지막으로, `show_json` 함수를 호출하여 업데이트된 보조기구의 정보를 JSON 형식으로 출력합니다.


[Assistant Files](https://platform.openai.com/files) 링크에서 직접 파일을 업로드하거나, 파일을 업로드하여 파일 ID를 얻을 수 있습니다.


In [ ]:
# 파일 ID를 불러옵니다
FILE_IDs = [
    "file-jW4Uwgad0wSdrlrQXXGCbbR1",  # ML 논문
    "file-JPNbRJacb6StlOpHNZNJANic",  # 2023년 경제 전망 보고서
]

In [ ]:
# 파일 업로드
file = client.files.create(
    file=open(
        "data/language_models_are_unsupervised_multitask_learners.pdf",
        "rb",
    ),
    purpose="assistants",
)

FILE_IDs = [file.id]

In [ ]:
# 보조기구 업데이트
assistant = client.beta.assistants.update(
    ASSISTANT_ID,
    tools=[{"type": "code_interpreter"}, {"type": "retrieval"}],
    file_ids=FILE_IDs,
)
show_json(assistant)

이 함수들은 머신러닝(ML) 논문 PDF에 대한 수학적 개념을 묻는 질문을 처리하기 위해 사용됩니다. `create_thread_and_run` 함수는 질문을 기반으로 쓰레드와 실행 객체를 생성합니다. 생성된 실행 객체는 `wait_on_run` 함수를 통해 해당 쓰레드의 실행이 완료될 때까지 대기합니다. 마지막으로, `get_response` 함수는 쓰레드로부터 응답을 받아와 `pretty_print` 함수를 사용하여 이를 예쁘게 출력합니다.


In [ ]:
run = ask(
    ASSISTANT_ID,
    THREAD_ID,
    "대한민국의 2023년 8월 경제전망을 pdf 파일에서 검색하여 알려줘.",
)

In [ ]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=THREAD_ID, run_id=run.id, order="asc"
)

for step in run_steps.data:
    # 각 단계의 세부 정보를 가져옵니다.
    step_details = step.step_details
    # 세부 정보를 JSON 형식으로 출력합니다.
    show_json(step_details)

In [ ]:
run = ask(
    ASSISTANT_ID,
    THREAD_ID,
    "What are some cool math concepts behind this ML paper pdf? Explain in two sentences.",
)

> **참고**
> 검색에는 [Annotations](https://platform.openai.com/docs/assistants/how-it-works/managing-threads-and-messages)와 같은 더 많은 복잡한 부분이 있으며, 이는 다른 쿡북에서 다룰 수 있습니다.


### 함수

최종적으로 강력한 도구로서, 여러분의 어시스턴트에게 사용자 정의 [함수](https://platform.openai.com/docs/assistants/tools/function-calling)를 지정할 수 있습니다(챗 완성 API에서의 [함수 호출](https://platform.openai.com/docs/guides/function-calling)과 매우 유사합니다). 실행 중에 어시스턴트는 지정한 하나 이상의 함수를 호출하고자 할 수 있습니다. 그러면 여러분이 함수를 호출하고 어시스턴트에게 출력을 제공할 책임이 있습니다.

예를 들어, 우리의 수학 튜터를 위한 `display_quiz()` 함수를 정의해 보겠습니다.

이 함수는 `title`과 `question`의 배열을 받아 퀴즈를 표시하고 사용자로부터 각각에 대한 입력을 받습니다:

- `title`
- `questions`
  - `question_text`
  - `question_type`: [`MULTIPLE_CHOICE`, `FREE_RESPONSE`]
  - `choices`: ["choice 1", "choice 2", ...]

유감스럽게도 Python 노트북 내에서 사용자 입력을 얻는 방법을 모르기 때문에, `get_mock_response...`를 사용하여 응답을 모방할 것입니다. 이것이 실제 사용자의 입력을 얻을 수 있는 부분입니다.


다양한 키즈 타입에 따라 사용자의 모크 답을 받아오는 함수들을 제공하고, 키즈를 표시하는 함수를 포함한 프로그램입니다. `get_mock_response_from_user_multiple_choice` 함수는 다짐 선택지 질문에 대한 모크 답을 반환하고, `get_mock_response_from_user_free_response` 함수는 자유 응답 질문에 대한 모크 답을 반환합니다. `display_quiz` 함수는 키즈의 제목과 질문들을 인자로 받아, 질문 타입에 따라 사용자의 모크 답을 수집하고 표시합니다. 다짐 선택지 질문에는 옵션들을 출력하고, 사용자는 모크 답을 선택합니다. 자유 응답 질문에는 답을 직접 입력받습니다.


In [ ]:
def display_quiz(title, questions):
    print("Quiz:", title)
    print()
    responses = []

    for q in questions:
        print(q["question_text"])
        response = ""

        # 다음으로, 다짐선택지 인쇄 옵션을 출력
        if q["question_type"] == "MULTIPLE_CHOICE":
            for i, choice in enumerate(q["choices"]):
                print(f"{i}. {choice}")
            response = input("정답을 선택하세요: ")

        # 그 외의 경우, 답을 받아옵니다.
        elif q["question_type"] == "FREE_RESPONSE":
            response = input("정답을 입력해 주세요: ")

        responses.append(response)
        print()

    return responses

In [ ]:
def display_quiz(title, questions):
    print("Quiz:", title)
    print()
    responses = []

    for q in questions:
        print(q["question_text"])
        response = ""

        # 다음으로, 다짐선택지 인쇄 옵션을 출력
        for i, choice in enumerate(q["choices"]):
            print(f"{i}. {choice}")
            response = input("정답을 선택하세요: ")

        responses.append(response)
        print()

    return responses

다음은 샘플 퀴즈의 모습입니다.


이 함수는 사용자에게 퀴즈를 표시하고, 사용자의 응답을 반환합니다. `display_quiz` 함수는 두 개의 매개변수를 받습니다: 첫 번째는 퀴즈의 제목이고, 두 번째는 질문 목록입니다. 질문 목록은 사전(dictionary)의 리스트로, 각 사전은 `question_text`와 `question_type` 키를 가지며, `question_type`이 `MULTIPLE_CHOICE`인 경우 `choices` 키도 포함합니다. `FREE_RESPONSE` 유형은 자유 응답을, `MULTIPLE_CHOICE` 유형은 다중 선택 문제를 나타냅니다. 함수의 실행 결과는 사용자의 응답을 담은 변수 `responses`에 저장되며, 이는 이후 `print` 함수를 통해 출력됩니다.


In [ ]:
responses = display_quiz(
    "Sample Quiz",
    [
        {"question_text": "What is your name?", "question_type": "FREE_RESPONSE"},
        {
            "question_text": "What is your favorite color?",
            "question_type": "MULTIPLE_CHOICE",
            "choices": ["Red", "Blue", "Green", "Yellow"],
        },
    ],
)
print("Responses:", responses)

이제, 이 함수의 인터페이스를 JSON 형식으로 정의해 보겠습니다. 그래서 우리의 Assistant가 호출할 수 있습니다:


이 JSON 구조는 `display_quiz` 함수의 구성을 설명합니다. 함수는 학생에게 퀴즈를 표시하고 학생의 응답을 반환하는 역할을 합니다. 퀴즈는 여러 개의 질문을 포함할 수 있으며, 각 질문은 텍스트와, 객관식일 경우 선택지를 포함할 수 있습니다. 이 구조는 퀴즈의 제목(`title`)과 질문들(`questions`)을 필수 요소로 요구하며, 각 질문은 질문 텍스트(`question_text`), 질문 유형(`question_type`), 그리고 선택지(`choices`)를 포함할 수 있습니다.


In [ ]:
function_json = {
    "name": "generate_quiz",
    "description": "Generate a quiz to the student, and returns the student's response. A single quiz has multiple questions.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "questions": {
                "type": "array",
                "description": "An array of questions, each with a title and multiple choice options.",
                "items": {
                    "type": "object",
                    "properties": {
                        "question_text": {"type": "string"},
                        "choices": {"type": "array", "items": {"type": "string"}},
                    },
                    "required": ["question_text"],
                },
            },
        },
        "required": ["title", "questions"],
    },
}

다시 한번, 대시보드나 API를 통해 우리의 Assistant를 업데이트합시다.


![Enabling custom function](./images/assistants_overview_enable_function.png)


> **참고**
> 대시보드에 함수 JSON을 붙여넣는 것이 들여쓰기 등으로 인해 조금 까다로웠습니다. 저는 ChatGPT에게 대시보드의 예시 중 하나와 동일하게 내 함수를 포맷하도록 요청했습니다 :).


이 함수는 `client` 객체의 `beta` 버전의 `assistants`를 업데이트하는 데 사용됩니다. `MATH_ASSISTANT_ID`를 사용하여 특정 수학 보조기구를 식별하고, 이를 업데이트하기 위해 여러 도구(`tools`)를 설정합니다. 이 도구들은 `code_interpreter`, `retrieval`, 그리고 사용자 정의 함수(`function`)를 포함합니다. 마지막으로, `show_json` 함수를 사용하여 업데이트된 보조기구의 정보를 JSON 형식으로 출력합니다.


In [ ]:
# OpenAI API를 사용하기 위한 클라이언트 객체를 생성합니다.
client = OpenAI(api_key=api_key)

# 파일 업로드
file = client.files.create(
    file=open(
        "data/SPRI_AI_Brief_2023년12월호.pdf",
        "rb",
    ),
    purpose="assistants",
)

FILE_IDs = [file.id]

# 퀴즈를 출제하는 역할을 하는 챗봇을 생성합니다.
# 이 챗봇은 간단한 문장이나 한 문장으로 질문에 답변합니다.
assistant = client.beta.assistants.create(
    name="Quiz Generator",
    instructions="You are a quiz generator. Generate quizzes for students to take based on the uploaded files.",
    model="gpt-4-turbo-preview",
    tools=[
        {"type": "retrieval"},
        {"type": "function", "function": function_json},
    ],
    file_ids=FILE_IDs,
)

ASSISTANT_ID = assistant.id
# 생성된 챗봇의 정보를 JSON 형태로 출력합니다.
show_json(assistant)

그리고 이제, 우리는 퀴즈를 요청합니다.


이 함수는 두 가지 유형의 질문을 포함한 퀴즈를 생성하고, 해당 퀴즈에 대한 응답에 대해 피드백을 요청하는 과정을 구현합니다. 첫 번째 단계에서는 `create_thread_and_run` 함수를 사용하여 퀴즈 생성과 피드백 요청 작업을 시작합니다. 이 작업은 개방형 질문과 객관식 질문 두 가지를 포함합니다. 다음으로, `wait_on_run` 함수를 통해 해당 작업의 완료를 기다립니다. 마지막으로, 작업의 상태를 확인하기 위해 `run.status`를 사용합니다.


In [ ]:
# 새로운 스레드를 생성합니다.
thread = client.beta.threads.create()
THREAD_ID = thread.id

# 사용자가 퀴즈를 만들도록 요청합니다.
run = ask(
    ASSISTANT_ID,
    THREAD_ID,
    # 두 가지 질문이 포함된 퀴즈를 만듭니다: 하나는 개방형, 다른 하나는 객관식입니다. 그런 다음, 응답에 대한 피드백을 제공합니다.
    "3개의 객관식 퀴즈(multiple choice questions)를 만들어 주세요. options: A, B, C, D"
    "퀴즈는 내가 업로드한 파일에 관한 내용이어야 합니다."
    "내가 제출한 responses에 대한 피드백을 주세요. "
    "내가 기입한 답, 정답, 제출한 답이 오답이라면 오답에 대한 피드백을 모두 포함해야 합니다. 출처가 있다면 페이지 번호를 표기해 주세요. "
    "피드백은 한글로 작성해 주세요. ",
)

In [ ]:
print(run.status)

이제, `Run`의 `status`를 확인할 때 `requires_action`이라고 나타납니다! 좀 더 자세히 살펴봅시다.


`required_action` 필드는 도구가 우리가 실행하고 그 출력을 어시스턴트에게 다시 제출하기를 기다리고 있음을 나타냅니다. 구체적으로, `display_quiz` 함수입니다! `name`과 `arguments`를 파싱하기 시작합시다.

> **참고**
> 이 경우에는 도구 호출이 하나뿐이라는 것을 알고 있지만, 실제로 어시스턴트는 여러 도구를 호출할 수 있습니다.


이 코드는 특정 실행(`run`)에서 요구되는 동작의 일부로 제출된 도구 호출 중 첫 번째를 추출합니다. 추출된 도구 호출에서 함수의 이름(`name`)과 인자(`arguments`)를 가져옵니다. 인자는 JSON 문자열로 저장되어 있으므로, 이를 파싱하기 위해 `json.loads` 함수를 사용합니다. 마지막으로, 함수의 이름과 인자를 출력합니다.


In [ ]:
# 단일 도구 호출 추출
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

이제 Assistant가 제공한 인자들로 `display_quiz` 함수를 실제로 호출해 봅시다:


이 함수는 `display_quiz`를 호출하여 퀴즈를 사용자에게 표시하고, 사용자의 응답을 수집합니다. `arguments` 딕셔너리에서 `title`과 `questions`을 인자로 사용합니다. 사용자의 응답은 `responses` 변수에 저장되며, 이후에 이를 출력합니다.


In [ ]:
def display_quiz(title, questions):
    print("Quiz:", title)
    print()
    responses = []

    for q in questions:
        print(q["question_text"])
        response = ""

        # 다음으로, 다짐선택지 인쇄 옵션을 출력
        for i, choice in enumerate(q["choices"]):
            print(f"{choice}")

        response = input("정답을 선택하세요: ")
        responses.append(response)
        print()

    return responses

In [ ]:
responses = display_quiz(arguments["title"], arguments["questions"])
# 퀴즈를 표시하고 사용자의 응답을 반환합니다.
print("Responses:", responses)

좋아요! (이 응답들은 우리가 이전에 모의한 것들입니다. 실제로는 이 함수 호출로부터 백엔드로부터 입력을 받게 될 것입니다.)

이제 우리의 응답들을 가지고 있으니, 이를 Assistant에 다시 제출해 봅시다. 우리는 이전에 파싱한 `tool_call`에서 찾을 수 있는 `tool_call` ID가 필요합니다. 또한, 우리의 응답 `list`를 `str`로 인코딩할 필요가 있습니다.


이 함수는 클라이언트의 `beta` 인터페이스를 사용하여 특정 스레드와 실행 ID에 대한 도구 출력을 제출합니다. `submit_tool_outputs` 메서드는 `thread_id`와 `run_id`를 인자로 받으며, `tool_outputs`는 도구 호출 ID(`tool_call_id`)와 해당 도구의 출력(`output`)을 포함하는 딕셔너리의 리스트입니다. 여기서 `output`은 JSON 형식으로 직렬화된 응답입니다. 마지막으로, `show_json` 함수는 제출된 도구 출력의 결과를 JSON 형태로 출력합니다.


In [ ]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=THREAD_ID,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(responses),
        }
    ],
)
show_json(run)

이제 다시 실행이 완료되기를 기다린 후, 우리의 Thread를 확인할 수 있습니다!


이 함수들은 스레드에서 특정 작업의 완료를 기다린 후, 그 결과를 예쁘게 출력하는 데 사용됩니다. `wait_on_run` 함수는 주어진 `run` 객체와 `thread`를 인자로 받아, 스레드에서 해당 작업이 완료될 때까지 대기합니다. 작업이 완료되면, `get_response` 함수를 통해 스레드의 응답을 가져오고, `pretty_print` 함수를 사용하여 이를 예쁘게 출력합니다.


In [ ]:
# 스레드에서 실행을 기다립니다.
run = wait_on_run(run, run.thread_id)
# 실행이 완료되면, 실행의 상태를 출력합니다.
if run.status == "completed":
    print("퀴즈를 제출했습니다.")
    # 전체 대화내용 출력
    print_message(get_response(THREAD_ID).data[-2:])

In [ ]:
print_message(get_response(THREAD_ID).data[-3:])

### 퀴즈 생성기 전체 코드


In [ ]:
# API KEY 정보를 불러옵니다
from dotenv import load_dotenv

load_dotenv()

1. 출제될 문제가 기반한 파일을 업로드합니다.
   1. 파일 업로드
   2. 기존 업로드된 File ID 사용


In [ ]:
# OpenAI API를 사용하기 위한 클라이언트 객체를 생성합니다.
client = OpenAI(api_key=api_key)

# 파일 업로드
file = client.files.create(
    file=open(
        "data/SPRI_AI_Brief_2023년12월호.pdf",
        "rb",
    ),
    purpose="assistants",
)

FILE_IDs = [file.id]

2. Assistant 를 생성합니다. 생성시 tools에 function을 추가합니다.


In [ ]:
function_json = {
    "name": "generate_quiz",
    "description": "Generate a quiz to the student, and returns the student's response. A single quiz has multiple questions.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "questions": {
                "type": "array",
                "description": "An array of questions, each with a title and multiple choice options.",
                "items": {
                    "type": "object",
                    "properties": {
                        "question_text": {"type": "string"},
                        "choices": {"type": "array", "items": {"type": "string"}},
                    },
                    "required": ["question_text"],
                },
            },
        },
        "required": ["title", "questions"],
    },
}

# 퀴즈를 출제하는 역할을 하는 챗봇을 생성합니다.
# 이 챗봇은 간단한 문장이나 한 문장으로 질문에 답변합니다.
assistant = client.beta.assistants.create(
    name="Quiz Generator",
    instructions="You are a quiz generator. Generate quizzes for students to take based on the uploaded files.",
    model="gpt-4-1106-preview",
    tools=[
        {"type": "retrieval"},
        {"type": "function", "function": function_json},
    ],
    file_ids=FILE_IDs,
)

ASSISTANT_ID = assistant.id

3. 새로운 스레드를 생성하고, 퀴즈를 출제합니다.
4. 퀴즈를 제출하고 피드백을 받습니다.


In [ ]:
# 새로운 스레드를 생성합니다.
thread = client.beta.threads.create()
THREAD_ID = thread.id

# 사용자가 퀴즈를 만들도록 요청합니다.
run = ask(
    ASSISTANT_ID,
    THREAD_ID,
    # 두 가지 질문이 포함된 퀴즈를 만듭니다: 하나는 개방형, 다른 하나는 객관식입니다. 그런 다음, 응답에 대한 피드백을 제공합니다.
    "5개의 객관식 퀴즈(multiple choice questions)를 만들어 주세요. options: A, B, C, D"
    "퀴즈는 내가 업로드한 파일에 관한 내용이어야 합니다."
    "내가 제출한 responses에 대한 피드백을 주세요. "
    "내가 기입한 답, 정답, 제출한 답이 오답이라면 오답에 대한 피드백을 모두 포함해야 합니다. "
    "피드백은 한글로 작성해 주세요. ",
)


def display_quiz(title, questions):
    print("Quiz:", title)
    print()
    responses = []

    for q in questions:
        print(q["question_text"])
        response = ""

        # 다음으로, 다짐선택지 인쇄 옵션을 출력
        for i, choice in enumerate(q["choices"]):
            print(f"{choice}")

        response = input("정답을 선택하세요: ")
        responses.append(response)
        print()

    return responses


if run.status == "requires_action":
    # 단일 도구 호출 추출
    tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
    name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    responses = display_quiz(arguments["title"], arguments["questions"])
    # 퀴즈를 표시하고 사용자의 응답을 반환합니다.
    print("Responses:", responses)

    run = client.beta.threads.runs.submit_tool_outputs(
        thread_id=THREAD_ID,
        run_id=run.id,
        tool_outputs=[
            {
                "tool_call_id": tool_call.id,
                "output": json.dumps(responses),
            }
        ],
    )

    # 스레드에서 실행을 기다립니다.
    run = wait_on_run(run, run.thread_id)
    # 실행이 완료되면, 실행의 상태를 출력합니다.
    if run.status == "completed":
        print("퀴즈를 제출했습니다.")
        # 전체 대화내용 출력
        print_message(get_response(THREAD_ID).data[-2:])

## 결론

이제 Code Interpreter, Retrieval, Functions와 같은 도구로 강력하고 상태를 유지하는 경험을 구축할 수 있는 탄탄한 기반을 마련했기를 바랍니다!

- [Annotations](https://platform.openai.com/docs/assistants/how-it-works/managing-threads-and-messages): 파일 인용 구문 분석
- [Files](https://platform.openai.com/docs/api-reference/assistants/file-object): 스레드 범위 대비 어시스턴트 범위
- [Parallel Function Calls](https://platform.openai.com/docs/guides/function-calling/parallel-function-calling): 단일 단계에서 여러 도구를 호출하기
